# Grupo 6
## Integrantes 
- Pablo Sarzosa
- Elisa Herrera
- Doménica Rodríguez 
- Andrés Albarracín

# 9.1. Unidades recurrentes cerradas (GRU)
 
En la Sección 8.7, analizamos cómo se calculan los gradientes en las RNN. En particular, encontramos que los productos largos de matrices pueden conducir a gradientes que se desvanecen o explotan. Pensemos brevemente en lo que significan tales anomalías de gradiente en la práctica:

Podríamos encontrarnos con una situación en la que una observación temprana sea muy significativa para predecir todas las observaciones futuras. Considere el caso algo artificial en el que la primera observación contiene una suma de verificación y el objetivo es discernir si la suma de verificación es correcta al final de la secuencia. En este caso, la influencia del primer token es vital. Nos gustaría tener algunos mecanismos para almacenar información temprana vital en una celda de memoria. Sin tal mecanismo, tendremos que asignar un gradiente muy grande a esta observación, ya que afecta a todas las observaciones posteriores.

Podríamos encontrarnos con situaciones en las que algunas fichas no lleven ninguna observación pertinente. Por ejemplo, al analizar una página web, puede haber un código HTML auxiliar que sea irrelevante para evaluar el sentimiento transmitido en la página. Nos gustaría tener algún mecanismo para omitir dichos tokens en la representación del estado latente.

Podríamos encontrarnos con situaciones en las que hay una ruptura lógica entre las partes de una secuencia. Por ejemplo, podría haber una transición entre los capítulos de un libro, o una transición entre un mercado de valores bajista y alcista. En este caso, sería bueno tener un medio para restablecer nuestra representación de estado interno.

Se han propuesto varios métodos para abordar esto. Uno de los primeros es la memoria a corto plazo [Hochreiter & Schmidhuber, 1997], que discutiremos en la Sección 9.2. La unidad recurrente cerrada (GRU) [Cho et al., 2014a] es una variante un poco más optimizada que a menudo ofrece un rendimiento comparable y es significativamente más rápida de calcular [Chung et al., 2014]. Por su sencillez, comencemos con el GRU.

## 9.1.1. Unidades Recurrentes Cerradas (GRU)

Como RNNs y particularmente la arquitectura LSTM (:numref:`sec_lstm`)
rápidamente ganó popularidad durante la década de 2010,
varios periódicos comenzaron a experimentar
con arquitecturas simplificadas con la esperanza
de retener la idea clave de incorporar
un estado interno y mecanismos de activación multiplicativos
pero con el objetivo de acelerar el cómputo.
La unidad recurrente cerrada (GRU) :cite:`Cho.Van-Merrienboer.Bahdanau.ea.2014`
ofreció una versión optimizada de la celda de memoria LSTM
que a menudo logra un rendimiento comparable
pero con la ventaja de ser más rápido
para calcular :cite:`Chung.Gulcehre.Cho.ea.2014`

### 9.1.1.1 Restablecer puerta y Actualizar puerta

Aquí, las tres puertas del LSTM se reemplazan por dos:
la *puerta de reinicio* y la *puerta de actualización*.
Al igual que con los LSTM, estas puertas reciben activaciones sigmoideas,
forzando sus valores a estar en el intervalo $(0, 1)$.
Intuitivamente, la puerta de reinicio controla cuánto del estado anterior
es posible que todavía queramos recordar.
Asimismo, una puerta de actualización nos permitiría controlar
cuánto del nuevo estado es solo una copia del antiguo estado.
:numref:`fig_gru_1` ilustra las entradas para ambos
las puertas de reinicio y actualización en una GRU,
dada la entrada del paso de tiempo actual
y el estado oculto del paso de tiempo anterior.
Las salidas de dos puertas se dan
por dos capas completamente conectadas
con una función de activación sigmoidea.

![Computing the reset gate and the update gate in a GRU model.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/gru-1.svg?raw=1)

Matemáticamente, para un paso de tiempo dado $t$,
supongamos que la entrada es un minilote
$\mathbf{X}_t \in \mathbb{R}^{n \veces d}$
(número de ejemplos: $n$, número de entradas: $d$)
y el estado oculto del paso de tiempo anterior
es $\mathbf{H}_{t-1} \in \mathbb{R}^{n \times h}$
(número de unidades ocultas: $h$).
Luego, la puerta de reinicio $\mathbf{R}_t \in \mathbb{R}^{n \times h}$
y la puerta de actualización $\mathbf{Z}_t \in \mathbb{R}^{n \times h}$ se calculan de la siguiente manera:

$$
\begin{aligned}
\mathbf{R}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xr} + \mathbf{H}_{t-1} \mathbf{W}_{hr} + \mathbf{b}_r),\\
\mathbf{Z}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xz} + \mathbf{H}_{t-1} \mathbf{W}_{hz} + \mathbf{b}_z),
\end{aligned}
$$

donde $\mathbf{W}_{xr}, \mathbf{W}_{xz} \in \mathbb{R}^{d \times h}$
y $\mathbf{W}_{hr}, \mathbf{W}_{hz} \in \mathbb{R}^{h \times h}$
son parámetros de peso y $\mathbf{b}_r, \mathbf{b}_z \in \mathbb{R}^{1 \times h}$
son parámetros de sesgo.

### 9.1.1.2. Candidato de estado oculto

A continuación, integramos la puerta de reinicio $\mathbf{R}_t$
con el mecanismo de actualización regular
en :eqref:`rnn_h_with_state`,
dando lugar a lo siguiente
*estado oculto del candidato*
$\tilde{\mathbf{H}}_t \in \mathbb{R}^{n \times h}$ en el paso de tiempo $t$:

$$\tilde{\mathbf{H}}_t = \tanh(\mathbf{X}_t \mathbf{W}_{xh} + \left(\mathbf{R}_t \odot \mathbf{H}_{ t-1}\right) \mathbf{W}_{hh} + \mathbf{b}_h),$$

donde $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$ y $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h ps
son parámetros de peso,
$\mathbf{b}_h \in \mathbb{R}^{1 \veces h}$
es el sesgo,
y el símbolo $\odot$ es el operador de producto de Hadamard (elemento).
Aquí usamos una función de activación tanh.

El resultado es un *candidato*, ya que todavía necesitamos
para incorporar la acción de la puerta de actualización.
Comparando con :eqref:`rnn_h_with_state`,
ahora la influencia de los estados anteriores
se puede reducir con la
multiplicación por elementos de
$\mathbf{R}_t$ y $\mathbf{H}_{t-1}$
en :eqref:`gru_tilde_H`.
Siempre que las entradas en la puerta de reinicio $\mathbf{R}_t$ estén cerca de 1,
recuperamos un RNN vainilla como en :eqref:`rnn_h_with_state`.
Para todas las entradas de la puerta de reinicio $\mathbf{R}_t$ que están cerca de 0,
el estado oculto candidato es el resultado de un MLP con $\mathbf{X}_t$ como entrada.
Cualquier estado oculto preexistente se *restablece* a los valores predeterminados.

:numref:`fig_gru_2` ilustra el flujo computacional después de aplicar la puerta de reinicio.

![Computing the candidate hidden state in a GRU model.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/gru-2.svg?raw=1)




### 9.1.1.3. Candidato de estado oculto


Finalmente, necesitamos incorporar el efecto de la puerta de actualización $\mathbf{Z}_t$.
Esto determina hasta qué punto el nuevo estado oculto $\mathbf{H}_t \in \mathbb{R}^{n \times h}$
coincide con el estado anterior $\mathbf{H}_{t-1}$ frente a cuánto
se parece al nuevo estado candidato $\tilde{\mathbf{H}}_t$.
La puerta de actualización $\mathbf{Z}_t$ puede usarse para este propósito,
simplemente tomando combinaciones convexas por elementos
de $\mathbf{H}_{t-1}$ y $\tilde{\mathbf{H}}_t$.
Esto lleva a la ecuación de actualización final para el GRU:

$$\mathbf{H}_t = \mathbf{Z}_t \odot \mathbf{H}_{t-1} + (1 - \mathbf{Z}_t) \odot \tilde{\mathbf{H}} _t.$$


Siempre que la puerta de actualización $\mathbf{Z}_t$ esté cerca de 1,
simplemente retenemos el antiguo estado.
En este caso se ignora la información de $\mathbf{X}_t$,
omitiendo efectivamente el paso de tiempo $t$ en la cadena de dependencia.
Por el contrario, siempre que $\mathbf{Z}_t$ esté cerca de 0,
el nuevo estado latente $\mathbf{H}_t$ se aproxima al estado latente candidato $\tilde{\mathbf{H}}_t$.
:numref:`fig_gru_3` ilustra el flujo computacional después de que la puerta de actualización está en acción.

![Cálculo del estado oculto en un modelo GRU.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/gru-3.svg?raw=1)


En resumen, las GRU tienen las siguientes dos características distintivas:

* Las puertas de reinicio ayudan a capturar dependencias a corto plazo en secuencias.
* Las puertas de actualización ayudan a capturar dependencias a largo plazo en secuencias.



## 9.1.2. Implementación desde cero
Para obtener una mejor comprensión del modelo GRU, implementémoslo desde cero. Comenzamos leyendo el conjunto de datos de la máquina del tiempo que usamos en la Sección 8.5. El código para leer el conjunto de datos se proporciona a continuación.

In [18]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use d2l;
use d2l::RNNModelScratch;
use d2l::RNNModel;
use d2l::Vocab;
use d2l::SeqDataLoader;
use d2l::Animator;
use d2l::Accumulator;
use d2l::Timer;
use AI::MXNet::Gluon qw(gluon);

IPerl->load_plugin('Chart::Plotly'); # Jupyter
use base("AI::MXNet::Gluon::Block");
#import Chart::Plotly 'show_plot'; # localmente

In [19]:
my ($batch_size, $num_steps) = (32, 35);
my ($train_iter, $vocab) = d2l->load_data_time_machine(batch_size      => $batch_size, 
                                                       num_steps       => $num_steps,
                                                       );

CODE(0xbd69140)Vocab=HASH(0xab8ddb0)

### 9.1.2.1. Inicialización de los parámetros del modelo
El siguiente paso es inicializar los parámetros del modelo. Extraemos los pesos de una distribución gaussiana con una desviación estándar de 0,01 y establecemos el sesgo en 0. El hiperparámetro num_hiddens define el número de unidades ocultas. Instanciamos todos los pesos y sesgos relacionados con la puerta de actualización, la puerta de reinicio, el estado oculto candidato y la capa de salida.

In [20]:
sub get_params{
    my ($vocab_size,$num_hiddens,$device)= @_;
    my $num_inputs = my $num_outputs = $vocab_size;
    
    sub normal{
    my ($shape, $device)= @_;
    
    print dump $shape;
        return mx->nd->random->normal(scale=>0.01, shape=>$shape, ctx=>$device);
        }
    
        
   sub three {
    my  ($num_inputs, $num_hiddens, $device) = @_;
    return normal(
            [$num_inputs, $num_hiddens], $device), normal(
                 [$num_inputs, $num_hiddens], $device), mx->nd->zeros([$num_hiddens], ctx => $device);
                 }

   
    # Hidden Layer parameters
    my ($W_xz, $W_hz, $b_z) = three($num_inputs, $num_hiddens, $device);
    my ($W_xr, $W_hr, $b_r) = three($num_inputs, $num_hiddens, $device);
    my ($W_xh, $W_hh, $b_h) = three($num_inputs, $num_hiddens, $device);
   
    # Output layer parameters
    
    my $W_hq = normal([$num_hiddens, $num_outputs]);
    my $b_q = mx->nd->zeros([$num_outputs], ctx => d2l->try_gpu());
    # Attach gradients
    #my $params = mx->nd->array(($W_xh, $W_hh, $b_h, $W_hq, $b_q));
    my $params = [$W_xz, $W_hz, $b_z, $W_xr, $W_hr, $b_r, $W_xh, $W_hh, $b_h, $W_hq, $b_q];
    for my $param (@{$params}){
        $param->attach_grad();
    }

    return $params;

}

Warning: Subroutine normal redefined at reply input line 5.

Subroutine three redefined at reply input line 13.

Subroutine get_params redefined at reply input line 1.


### 9.1.2.2. Definición del modelo

Ahora definiremos la función de inicialización de estado oculto init_gru_state. Al igual que la función init_rnn_state definida en la Sección 8.5, esta función devuelve un tensor con una forma (tamaño del lote, número de unidades ocultas) cuyos valores son todos ceros.Definición del modelo



In [22]:
sub init_gru_state{
    my ($batch_size, $num_hiddens, $device)=@_;
    return mx->nd->zeros([$batch_size,$num_hiddens],ctx=>$device);
}


Warning: Subroutine init_gru_state redefined at reply input line 1.


Ahora estamos listos para definir el modelo GRU. Su estructura es la misma que la de la celda RNN básica, excepto que las ecuaciones de actualización son más complejas.

In [23]:
sub gru {
    my ($inputs, $state, $params) = @_;
    # Shape of inputs: (num_steps, batch_size, vocab_size)
    my ($W_xz, $W_hz, $b_z, $W_xr, $W_hr, $b_r, $W_xh, $W_hh, $b_h, $W_hq, $b_q ) = @{$params};
    my ($H) = @$state;
    my @outputs = ();
# Shape of X: (batch_size, vocab_size)
        for my $X (@{$inputs}) {
            my $Z = mx->nd->sigmoid(mx->nd->dot($X, $W_xz) + mx->nd->dot($H, $W_hz) + $b_z);
            my $R = mx->nd->sigmoid(mx->nd->dot($X, $W_xr) + mx->nd->dot($H, $W_hr) + $b_r);
            my $H_tilda = mx->nd->tanh(mx->nd->dot($X, $W_xh) + mx->nd->dot($R * $H, $W_hh) + $b_h);
            $H = $Z * $H + (1 - $Z) * $H_tilda;
            my $Y = mx->nd->dot($H, $W_hq) + $b_q;
            #print @outputs,": ";
            push @outputs, ($Y);
}
    #print @outputs->len;
    return mx->nd->concat(@outputs, axis=>0, ($H));
   
}

Warning: Subroutine gru redefined at reply input line 1.


### 9.1.2.3. Entrenamiento y Predicción
El entrenamiento y la predicción funcionan exactamente de la misma manera que en la Sección 8.5. Después del entrenamiento, imprimimos la perplejidad en el conjunto de entrenamiento y la secuencia predicha siguiendo los prefijos proporcionados "viajero en el tiempo" y "viajero", respectivamente.

In [29]:
my ($vocab_size, $num_hiddens, $device) =($vocab->len, 256, d2l->try_gpu());
my $num_epochs = 500;
my $lr =1;
my $model = RNNModelScratch->new($vocab->len, $num_hiddens, $device, \&get_params, \&init_gru_state, \&gru);


[28, 256][28, 256][28, 256][28, 256][28, 256][28, 256][256, 28]

RNNModelScratch=HASH(0xbd3e5b0)

In [11]:
d2l->train_ch8($model, $train_iter, vocab, $lr, num_epochs, device);

Error: Bareword "vocab" not allowed while "strict subs" in use at reply input line 1.
Bareword "num_epochs" not allowed while "strict subs" in use at reply input line 1.
Bareword "device" not allowed while "strict subs" in use at reply input line 1.



## 9.1.3. Implementación concisa
En las API de alto nivel, podemos instanciar directamente un modelo de GPU. Esto encapsula todos los detalles de configuración que hicimos explícitos anteriormente. El código es significativamente más rápido ya que usa operadores compilados en lugar de Python para muchos detalles que explicamos antes.

In [13]:
my $gru_layer = rnn->GRU($num_hiddens);
my $model = RNNModel->new($gru_layer, ($vocab)->len);

Dense(28 -> 0, linear)

RNNModel(

)

In [15]:

d2l->train_ch8($model, $train_iter, vocab, $lr, $num_epochs, $device);

Error: Bareword "vocab" not allowed while "strict subs" in use at reply input line 2.



## 9.1.4 Resumen

En comparación con las LSTM, las GRU logran un rendimiento similar, pero tienden a ser más livianas desde el punto de vista computacional.
En general, en comparación con los RNN simples, los RNN controlados como LSTM y GRU
puede capturar mejor las dependencias para secuencias con grandes distancias de paso de tiempo.
Las GRU contienen RNN básicos como su caso extremo cada vez que se activa la puerta de reinicio.
También pueden omitir subsecuencias activando la puerta de actualización.


## 9.1.5 Ejercicios

1. Suponga que solo queremos usar la entrada en el paso de tiempo $t'$ para predecir la salida en el paso de tiempo $t > t'$. ¿Cuáles son los mejores valores para las puertas de reinicio y actualización para cada paso de tiempo?
1. Ajuste los hiperparámetros y analice su influencia en el tiempo de ejecución, la perplejidad y la secuencia de salida.
1. Compare el tiempo de ejecución, la perplejidad y las cadenas de salida para las implementaciones `rnn.RNN` y `rnn.GRU` entre sí.
1. ¿Qué sucede si implementa solo partes de una GRU, por ejemplo, con solo una puerta de reinicio o solo una puerta de actualización?